In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import os, re, sys
from sklearn.model_selection import train_test_split
import skimage.io
import gc

In [2]:
# test if GPUs are available
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# set project root, maybe you need to firstly 
# add shortcut of CS 766 Project to drive.
project_root = './drive/MyDrive/CS 766 Project/Project Coding and Data Files'

In [9]:
# class to initialize CNNs
class OriginCNN(object):
  """docstring for OriginCNN"""
  def __init__(self):
    # self.optimizer = 'sgd'
    self.optimizer = tf.keras.optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
    self.loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # will add more properties


  def build(self):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), strides=(3,3), activation='relu', input_shape=(1000, 1000, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), strides=(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), strides=(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='sigmoid'))
    model.add(layers.Dense(5))

    model.summary()
    model.compile(optimizer=self.optimizer,
                loss=self.loss,
                metrics=['accuracy'])
    self.model = model


  def train(self, train_images, train_labels, validate_images, validate_labels, epochs):
    self.history = self.model.fit(train_images, train_labels, epochs=epochs, 
        validation_data=(validate_images, validate_labels))


  def evaluate(self, test_images, test_labels):
    _, test_acc = self.model.evaluate(test_images, test_labels, verbose=2)
    return test_images.shape[0], test_acc

  def save_model(self, filepath):
    self.model.save(filepath)

  def load_model(self, filepath):
    self.model = tf.keras.models.load_model(filepath)

  def extract_feature(self, images):
    extract = models.Model(self.model.inputs, self.model.layers[-3].output)
    return extract.predict(images)

In [14]:
class DR_resized(object):
  """docstring for DR_resized"""
  def __init__(self, is_training, batch_id):
    if(is_training):
      # load images
      all_images = np.load(os.path.join(project_root, "Processed_Data_Batches", "Even_Class_Distribution_Datasets", "Processed_Xtrain_Batch%d_even.npy" % batch_id))
      # add a dimension for channels
      all_images = np.expand_dims(all_images, axis=-1)

      # load labels
      all_labels = np.load(os.path.join(project_root, "Data Batches", "Even Class Distribution Datasets", "Ytrain_Batch%d_even.npy" % batch_id), allow_pickle=True)
      all_labels = np.array([item[1] for item in all_labels])
      all_labels = np.expand_dims(all_labels, axis=-1)
      
      self.train_images, self.validate_images, self.train_labels, self.validate_labels \
      = train_test_split(all_images, all_labels, test_size=0.2, random_state=9876)
    else:
        # load images
      all_images = np.load(os.path.join(project_root, "Processed_Data_Batches", "Even_Class_Distribution_Datasets", "Processed_Xtest_Batch%d_even.npy") % batch_id)
      # add a dimension for channels
      all_images = np.expand_dims(all_images, axis=-1)

      # load labels
      all_labels = np.load(os.path.join(project_root, "Data Batches", "Even Class Distribution Datasets", "Ytest_Batch%d_even.npy" % batch_id), allow_pickle=True)
      all_labels = np.array([item[1] for item in all_labels])
      all_labels = np.expand_dims(all_labels, axis=-1)
      
      self.test_images = all_images
      self.test_labels = all_labels
  
  def clear(self):
    self.train_images = None
    self.validate_images = None
    self.test_images = None
    self.train_labels = None
    self.validate_labels = None
    self.test_labels = None


In [7]:
# initialize CNN
myModel = OriginCNN()
myModel.build()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 333, 333, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 166, 166, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 55, 55, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0

In [15]:
# training process
file_num = 9
epoch_num = 5
for i in range(file_num):
  myData = DR_resized(True, i)
  print("data batch %d loaded" % i)
  myModel.train(myData.train_images, myData.train_labels, myData.validate_images, myData.validate_labels, epoch_num)
  print("data batch %d trained" % i)
  myData.clear()
  gc.collect()

data batch 0 loaded
Epoch 1/5
8/8 [==============================] - 33s 122ms/step - loss: 1.7316 - accuracy: 0.2242 - val_loss: 1.6324 - val_accuracy: 0.1935
Epoch 2/5
8/8 [==============================] - 1s 73ms/step - loss: 1.6456 - accuracy: 0.1545 - val_loss: 1.6426 - val_accuracy: 0.1774
Epoch 3/5
8/8 [==============================] - 1s 70ms/step - loss: 1.6220 - accuracy: 0.1785 - val_loss: 1.6146 - val_accuracy: 0.1935
Epoch 4/5
8/8 [==============================] - 1s 70ms/step - loss: 1.6197 - accuracy: 0.1742 - val_loss: 1.6177 - val_accuracy: 0.1774
Epoch 5/5
8/8 [==============================] - 1s 69ms/step - loss: 1.6165 - accuracy: 0.2139 - val_loss: 1.6323 - val_accuracy: 0.1774
data batch 0 trained
data batch 1 loaded
Epoch 1/5
8/8 [==============================] - 1s 82ms/step - loss: 1.6254 - accuracy: 0.2016 - val_loss: 1.6206 - val_accuracy: 0.1774
Epoch 2/5
8/8 [==============================] - 1s 72ms/step - loss: 1.6142 - accuracy: 0.2016 - val_loss: 1

In [ ]:
# testing process
file_num = 2
num_list = []
acc_list = []
for i in range(file_num):
  myData = DR_resized(False, i)
  num, acc = myModel.evaluate(myData.test_images, myData.test_labels)
  print(num, acc)
  num_list.append(num)
  acc_list.append(acc)
  myData.clear()
  gc.collect()

98/98 - 3s - loss: 0.8625 - accuracy: 0.7354
3118 0.735407292842865
123/123 - 4s - loss: 0.8743 - accuracy: 0.7312
3928 0.7311608791351318


In [ ]:
# compute overall accuracy
num_list = np.array(num_list)
acc_list = np.array(acc_list)
acc_overall = (num_list * acc_list).sum() / num_list.sum()
print(acc_overall)

0.733040004587972


In [ ]:
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y %H:%M:%S")
print("date and time =", dt_string)
myModel.save_model(os.path.join(project_root, "Trained Models", dt_string))

date and time = 16-03-2021 01:13:01
INFO:tensorflow:Assets written to: ./drive/MyDrive/CS 766 Project/Project Coding and Data Files/Trained Models/16-03-2021 01:13:01/assets


In [ ]:
# this part is for loading parameters and extracting features
# load parameters
parameter_version = "16-03-2021 01:13:01"
if not os.path.exists(os.path.join(project_root, "features", parameter_version)):
    os.mkdir(os.path.join(project_root, "features", parameter_version))

my_model_pretrained = OriginCNN()
my_model_pretrained.build()
my_model_pretrained.load_model(os.path.join(project_root, "Trained Models", parameter_version))

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 333, 333, 32)      320       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 166, 166, 32)      0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 55, 55, 32)        9248      
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 512)             

In [ ]:
# load training data, extract features and save to npy files
file_num = 9
for i in range(file_num):
  myData = DR_resized(True, i)
  feature = my_model_pretrained.extract_feature(myData.train_images)
  np.save(os.path.join(project_root, "features", parameter_version, "Xtrain_feature%d.npy" % i), feature)
  print("Xtrain_feature%d.npy" % i)
  myData.clear()
  gc.collect()

# load testing data, extract features and save to npy files
file_num = 2
for i in range(file_num):
  myData = DR_resized(False, i)
  feature = my_model_pretrained.extract_feature(myData.test_images)
  np.save(os.path.join(project_root, "features", parameter_version, "Xtest_feature%d.npy" % i), feature)
  print("Xtest_feature%d.npy" % i)
  myData.clear()
  gc.collect()

Xtrain_feature0.npy
Xtrain_feature1.npy
Xtrain_feature2.npy
Xtrain_feature3.npy
Xtrain_feature4.npy
Xtrain_feature5.npy
Xtrain_feature6.npy
Xtrain_feature7.npy
Xtrain_feature8.npy
Xtest_feature0.npy
Xtest_feature1.npy
